In [1]:
import xarray as xr
import os
import tensorflow as tf
import jdutil # ./jdutil.py
from util import *  # ./util.py
import datetime
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import numpy as np
import math
from sklearn.model_selection import train_test_split

print('TensorFlow version:',tf.__version__)

TensorFlow version: 2.10.0


In [57]:
atms_npp = xr.load_dataset('.\\data\\atms_npp_surface_contribution.nc')
atms_npp = atms_npp.drop_sel(channel=[8,9,10,11,12,13,14,15])
arr = atms_npp['surface_contribution'].values = np.nan_to_num(atms_npp['surface_contribution'].values,nan=1)
arr[arr < 0] = 0
arr[arr > 1] = 1

atms_npp = atms_npp.where(atms_npp.lsm == 1.0, drop=True)
atms_npp = atms_npp.where(atms_npp.tcc <= 0.1, drop=True)
atms_npp

<xarray.Dataset>
Dimensions:               (level: 137, obs_id: 1193, channel: 14)
Coordinates:
  * level                 (level) int64 1 2 3 4 5 6 ... 132 133 134 135 136 137
    jd2                   float64 2.458e+06
  * channel               (channel) int64 1 2 3 4 5 6 7 16 17 18 19 20 21 22
    lat2                  (obs_id) float64 68.4 68.4 68.4 ... 23.4 23.1 23.1
    lon2                  (obs_id) float64 -166.2 -165.9 ... -109.8 -106.2
Dimensions without coordinates: obs_id
Data variables: (12/151)
    crwc                  (level, obs_id) float64 -5.821e-11 ... -5.821e-11
    cswc                  (level, obs_id) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    t                     (level, obs_id) float64 170.4 170.3 ... 302.6 305.4
    q                     (level, obs_id) float64 2.429e-06 ... 0.01498
    o3                    (level, obs_id) float64 1.131e-07 ... 5.596e-08
    clwc                  (level, obs_id) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    ...                    ...
    scan_line_stddev      (obs_id) float64 0.7071 nan nan ... nan 0.7071 0.7071
    scan_line_count       (obs_id) float64 2.0 1.0 1.0 2.0 ... 1.0 1.0 2.0 2.0
    fov1                  (obs_id) float64 9.0 9.0 10.0 9.0 ... 94.0 86.0 94.0
    fov1_stddev           (obs_id) float64 0.0 nan nan ... nan 4.767e-06 0.0
    fov1_count            (obs_id) float64 2.0 1.0 1.0 2.0 ... 1.0 1.0 2.0 2.0
    surface_contribution  (obs_id, channel) float64 0.6102 0.6098 ... 1.0 1.0

In [58]:
atms_n20 = xr.load_dataset('.\\data\\atms_n20_surface_contribution.nc')
atms_n20 = atms_n20.drop_sel(channel=[8,9,10,11,12,13,14,15])
arr = atms_n20['surface_contribution'].values = np.nan_to_num(atms_n20['surface_contribution'].values,nan=1)
arr[arr < 0] = 0
arr[arr > 1] = 1

atms_n20 = atms_n20.where(atms_n20.lsm == 1.0, drop=True)
atms_n20 = atms_n20.where(atms_n20.tcc <= 0.1, drop=True)
atms_n20

<xarray.Dataset>
Dimensions:               (level: 137, obs_id: 7733, channel: 14)
Coordinates:
  * level                 (level) int64 1 2 3 4 5 6 ... 132 133 134 135 136 137
    jd2                   float64 2.458e+06
  * channel               (channel) int64 1 2 3 4 5 6 7 16 17 18 19 20 21 22
    lat2                  (obs_id) float64 -6.0 -6.0 -6.0 ... -87.0 -87.0 -87.0
    lon2                  (obs_id) float64 -68.1 -67.8 -67.5 ... -91.2 -90.9
Dimensions without coordinates: obs_id
Data variables: (12/151)
    crwc                  (level, obs_id) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    cswc                  (level, obs_id) float64 -2.328e-10 ... -2.328e-10
    t                     (level, obs_id) float64 188.4 188.3 ... 227.7 227.8
    q                     (level, obs_id) float64 2.018e-06 ... 5.326e-05
    o3                    (level, obs_id) float64 1.757e-07 ... 6.389e-08
    clwc                  (level, obs_id) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    ...                    ...
    scan_line_stddev      (obs_id) float64 0.5523 1.502e-05 ... nan nan
    scan_line_count       (obs_id) float64 3.0 2.0 2.0 1.0 ... 2.0 2.0 1.0 1.0
    fov1                  (obs_id) float64 67.3 65.5 63.57 ... 10.0 10.0 10.0
    fov1_stddev           (obs_id) float64 0.5653 0.7071 0.7071 ... nan nan
    fov1_count            (obs_id) float64 3.0 2.0 2.0 1.0 ... 2.0 2.0 1.0 1.0
    surface_contribution  (obs_id, channel) float64 0.9337 0.9327 ... 0.5763

In [59]:
variables = [
        'lat1',
        'lon1',
        'zenith_angle',
        'azimuth_angle',
        'skt',
        'u10n',
        'v10n',
        'u10',
        'v10',
        'sp',
        'msl',
        'fsr',
        'u100',
        'v100',
        'cvl',
        'cvh',
        'tvl',
        'tvh',
        'swvl1',
        'swvl2',
        'swvl3',
        'swvl4',
        'slt',
        'lai_lv',
        'lai_hv',
        'sdfor',
        'stl1',
        'stl2',
        'stl3',
        'stl4',
        'd2m'
    ]

x_train = atms_n20[variables].to_array().values.T
x_test  = atms_npp[variables].to_array().values.T
y_train = atms_n20['surface_contribution'].values
y_test  = atms_npp['surface_contribution'].values

print('max y_train', max(y_train.flatten()))
print('min y_train', min(y_train.flatten()))
print('max y_test', max(y_test.flatten()))
print('min y_test', min(y_test.flatten()))

x_train_mean = x_train.mean(axis=0)
x_train_std  = x_train.std(axis=0)
y_train_mean = y_train.mean(axis=0)

x_train = (x_train - x_train_mean) / x_train_std
x_test = (x_test - x_train_mean) / x_train_std
y_train = y_train - y_train_mean
y_test = y_test - y_train_mean

max y_train 1.0
min y_train 0.0
max y_test 1.0
min y_test 0.0


In [60]:
# train normally

model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(x_train.shape[1]),
            tf.keras.layers.Dense(10, activation='relu'),
            tf.keras.layers.Dense(35, activation='relu'),
            tf.keras.layers.Dense(10, activation='relu'),
            tf.keras.layers.Dense(20, activation='relu'),
            tf.keras.layers.Dense(y_train.shape[1])
        ])

loss_fn = tf.keras.losses.MeanAbsoluteError()

model.compile(optimizer='adam',
              loss=loss_fn)

hist = model.fit(x_train, y_train, 
          epochs=40,
          validation_data=(x_test, y_test)
       )

Epoch 1/40
242/242 [==============================] - 2s 5ms/step - loss: 0.0994 - val_loss: 0.1529
Epoch 2/40
242/242 [==============================] - 1s 4ms/step - loss: 0.0478 - val_loss: 0.1111
Epoch 3/40
242/242 [==============================] - 1s 4ms/step - loss: 0.0376 - val_loss: 0.1090
Epoch 4/40
242/242 [==============================] - 1s 4ms/step - loss: 0.0345 - val_loss: 0.1078
Epoch 5/40
242/242 [==============================] - 1s 4ms/step - loss: 0.0330 - val_loss: 0.1136
Epoch 6/40
242/242 [==============================] - 1s 4ms/step - loss: 0.0319 - val_loss: 0.1131
Epoch 7/40
242/242 [==============================] - 1s 4ms/step - loss: 0.0312 - val_loss: 0.1161
Epoch 8/40
242/242 [==============================] - 1s 4ms/step - loss: 0.0304 - val_loss: 0.1133
Epoch 9/40
242/242 [==============================] - 1s 4ms/step - loss: 0.0297 - val_loss: 0.1118
Epoch 10/40
242/242 [==============================] - 1s 5ms/step - loss: 0.0289 - val_loss: 0.1089

In [70]:
model.save("surface_contribution_model_3hidden50.h5")
new_model = tf.keras.models.load_model("surface_contribution_model_3hidden50.h5")
new_model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_61 (Dense)            (None, 31)                992       
                                                                 
 dense_62 (Dense)            (None, 50)                1600      
                                                                 
 dropout_6 (Dropout)         (None, 50)                0         
                                                                 
 dense_63 (Dense)            (None, 50)                2550      
                                                                 
 dropout_7 (Dropout)         (None, 50)                0         
                                                                 
 dense_64 (Dense)            (None, 50)                2550      
                                                                 
 dropout_8 (Dropout)         (None, 50)              

In [41]:
val_loss = model.evaluate(x_test, y_test)
print('val loss',val_loss)
train_loss = model.evaluate(x_train, y_train)
print('train loss',train_loss)
y_pred = model.predict(x_test)
print('average error in each variable',abs(y_test - y_pred).mean())
y_pred_unnormalized = y_pred + y_train_mean

ds = xr.Dataset(
    data_vars=dict(prediction=(["obs_id", "channel"], y_pred_unnormalized))
)
ds.to_netcdf("atms_npp_surface_contribution_predictions.nc")

val loss 0.02849419042468071
38/38 [==============================] - 0s 2ms/step - loss: 0.0285
train loss 0.02849419042468071
38/38 [==============================] - 0s 1ms/step
average error in each variable 0.028494187094210713


In [43]:
# test model from loaded weights

current_variables = [4, 10 ,15, 27]

x_train_subset = x_train[:,current_variables]
x_test_subset = x_test[:,current_variables]

model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(x_train_subset.shape[1]),
            tf.keras.layers.Dense(30, activation='relu'),
            tf.keras.layers.Dense(22)
        ])

loss_fn = tf.keras.losses.MeanSquaredError()

model.compile(optimizer='adam',
              loss=loss_fn)
model.build(input_shape=(None,4))
model.load_weights('.\\checkpoints\\shallow_h30[4, 10, 15, 27].h5')
loss = model.evaluate(x_test_subset, y_test)
print('loss',loss)
y_pred = model.predict(x_test_subset)
print('average error in each variable',abs(y_test - y_pred).mean())
y_pred_unnormalized = y_pred + y_train_mean
y_pred_unnormalized

38/38 [==============================] - 0s 1ms/step - loss: 74.2951
loss 74.29512023925781
38/38 [==============================] - 0s 1ms/step
average error in each variable 6.28922325935255


array([[268.49682017, 266.01462367, 270.07175542, ..., 265.41046595,
        259.08116687, 253.39722832],
       [266.96065303, 264.59486583, 268.91334249, ..., 264.82364726,
        258.67306674, 253.15303334],
       [265.53070613, 263.27310374, 267.83790494, ..., 264.27115893,
        258.28557552, 252.91893872],
       ...,
       [287.94321605, 284.22502711, 284.87660695, ..., 273.81030154,
        265.33366359, 257.59867486],
       [286.14388438, 282.74547389, 282.98639394, ..., 275.33169627,
        267.09293521, 259.42340668],
       [288.6281335 , 284.89311793, 285.34534551, ..., 274.4826076 ,
        265.8982159 , 258.07776555]])

In [48]:
ds = xr.Dataset(
    data_vars=dict(prediction=(["obs_id", "channel"], y_pred_unnormalized))
)
ds.to_netcdf("atms_npp_predictions.nc")

In [35]:
test_model_architecture_losses = {}
best_loss = 999999

In [44]:
# find good model by testing various model architectures

#layer_5 = tf.keras.layers.Dense(5, activation='relu')
layer_10 = tf.keras.layers.Dense(10, activation='relu')
#layer_15 = tf.keras.layers.Dense(15, activation='relu')
layer_20 = tf.keras.layers.Dense(20, activation='relu')
layer_35 = tf.keras.layers.Dense(35, activation='relu')
layer_50 = tf.keras.layers.Dense(50, activation='relu')

layers = [10, 20, 35, 50]
combos = combinations(layers, 4)

# training cycle
loss_fn = tf.keras.losses.MeanAbsoluteError()

for lst in combos:
    # build and compile model
    model = tf.keras.models.Sequential([tf.keras.layers.Dense(x_train.shape[1], input_shape=(x_train.shape[1],))])
    index = ""
    for layer in lst:
        if layer == 10:
            index += "10_"
            model.add(tf.keras.layers.Dense(10, activation='relu'))
        elif layer == 20:
            index += "20_"
            model.add(tf.keras.layers.Dense(20, activation='relu'))
        elif layer == 35:
            index += "35_"
            model.add(tf.keras.layers.Dense(35, activation='relu'))
        else:
            index += "50_"
            model.add(tf.keras.layers.Dense(50, activation='relu'))
        
    model.add(tf.keras.layers.Dense(y_train.shape[1]))

    model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
            filepath='checkpoints\\model_architecture_' + index + '.h5',
            save_weights_only=False,
            save_best_only=True,
            verbose=0
        )

    early_stopping = tf.keras.callbacks.EarlyStopping(
        patience=5
    )

    model.compile(optimizer='adam',
                  loss=loss_fn
                 )


    print('fitting model with index',index)
    hist = model.fit(x_train, y_train, 
              epochs=50,
              validation_data=(x_test, y_test),
              callbacks=[model_checkpoint, early_stopping]
           )

    current_loss = min(hist.history['val_loss'])
    test_model_architecture_losses[index] = current_loss
    print('loss for',index,current_loss)

    if current_loss < best_loss:
        best_loss = current_loss
        print('new best loss:',index, best_loss)

fitting model with index 10_
Epoch 1/50
242/242 [==============================] - 2s 4ms/step - loss: 0.1351 - val_loss: 0.2590
Epoch 2/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0653 - val_loss: 0.2031
Epoch 3/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0519 - val_loss: 0.1787
Epoch 4/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0435 - val_loss: 0.1778
Epoch 5/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0384 - val_loss: 0.1662
Epoch 6/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0360 - val_loss: 0.1719
Epoch 7/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0344 - val_loss: 0.1607
Epoch 8/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0335 - val_loss: 0.1708
Epoch 9/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0328 - val_loss: 0.1799
Epoch 10/50
242/242 [==============================] - 1s 3ms/step - lo

loss for 10_10_ 0.09538006037473679
new best loss: 10_10_ 0.09538006037473679
fitting model with index 20_10_
Epoch 1/50
242/242 [==============================] - 2s 5ms/step - loss: 0.1180 - val_loss: 0.1998
Epoch 2/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0653 - val_loss: 0.1820
Epoch 3/50
242/242 [==============================] - 1s 4ms/step - loss: 0.0498 - val_loss: 0.1452
Epoch 4/50
242/242 [==============================] - 1s 4ms/step - loss: 0.0412 - val_loss: 0.1378
Epoch 5/50
242/242 [==============================] - 1s 4ms/step - loss: 0.0362 - val_loss: 0.1303
Epoch 6/50
242/242 [==============================] - 1s 4ms/step - loss: 0.0348 - val_loss: 0.1274
Epoch 7/50
242/242 [==============================] - 1s 4ms/step - loss: 0.0339 - val_loss: 0.1227
Epoch 8/50
242/242 [==============================] - 1s 4ms/step - loss: 0.0332 - val_loss: 0.1202
Epoch 9/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0327 - val_los

242/242 [==============================] - 1s 4ms/step - loss: 0.0318 - val_loss: 0.1296
Epoch 7/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0308 - val_loss: 0.1374
Epoch 8/50
242/242 [==============================] - 1s 4ms/step - loss: 0.0299 - val_loss: 0.1252
Epoch 9/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0292 - val_loss: 0.1291
Epoch 10/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0285 - val_loss: 0.1296
Epoch 11/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0277 - val_loss: 0.1308
Epoch 12/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0272 - val_loss: 0.1241
Epoch 13/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0267 - val_loss: 0.1344
Epoch 14/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0262 - val_loss: 0.1271
Epoch 15/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0258 - val_loss: 0.1245
Epoch

242/242 [==============================] - 1s 4ms/step - loss: 0.0435 - val_loss: 0.1289
Epoch 3/50
242/242 [==============================] - 1s 4ms/step - loss: 0.0357 - val_loss: 0.1254
Epoch 4/50
242/242 [==============================] - 1s 4ms/step - loss: 0.0320 - val_loss: 0.1229
Epoch 5/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0298 - val_loss: 0.1190
Epoch 6/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0282 - val_loss: 0.1250
Epoch 7/50
242/242 [==============================] - 1s 4ms/step - loss: 0.0268 - val_loss: 0.1228
Epoch 8/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0258 - val_loss: 0.1244
Epoch 9/50
242/242 [==============================] - 1s 4ms/step - loss: 0.0250 - val_loss: 0.1233
Epoch 10/50
242/242 [==============================] - 1s 4ms/step - loss: 0.0243 - val_loss: 0.1203
loss for 35_35_ 0.11901122331619263
fitting model with index 50_35_
Epoch 1/50
242/242 [======================

242/242 [==============================] - 1s 2ms/step - loss: 0.0213 - val_loss: 0.1358
loss for 20_50_ 0.12475568056106567
fitting model with index 35_50_
Epoch 1/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0941 - val_loss: 0.1534
Epoch 2/50
242/242 [==============================] - 1s 2ms/step - loss: 0.0414 - val_loss: 0.1417
Epoch 3/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0343 - val_loss: 0.1380
Epoch 4/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0308 - val_loss: 0.1353
Epoch 5/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0283 - val_loss: 0.1271
Epoch 6/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0266 - val_loss: 0.1251
Epoch 7/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0254 - val_loss: 0.1241
Epoch 8/50
242/242 [==============================] - 1s 2ms/step - loss: 0.0246 - val_loss: 0.1296
Epoch 9/50
242/242 [=======================

...

242/242 [==============================] - 1s 3ms/step - loss: 0.0260 - val_loss: 0.1269
Epoch 5/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0244 - val_loss: 0.1278
Epoch 6/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0233 - val_loss: 0.1308
Epoch 7/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0227 - val_loss: 0.1321
Epoch 8/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0221 - val_loss: 0.1308
Epoch 9/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0212 - val_loss: 0.1315
loss for 35_50_35_50_ 0.12686902284622192
fitting model with index 50_50_35_50_
Epoch 1/50
242/242 [==============================] - 2s 4ms/step - loss: 0.0638 - val_loss: 0.1235
Epoch 2/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0309 - val_loss: 0.1116
Epoch 3/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0273 - val_loss: 0.1072
Epoch 4/50
242/242 [===========

Epoch 2/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0335 - val_loss: 0.1379
Epoch 3/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0291 - val_loss: 0.1411
Epoch 4/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0264 - val_loss: 0.1371
Epoch 5/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0249 - val_loss: 0.1239
Epoch 6/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0238 - val_loss: 0.1206
Epoch 7/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0232 - val_loss: 0.1153
Epoch 8/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0223 - val_loss: 0.1166
Epoch 9/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0219 - val_loss: 0.1209
Epoch 10/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0215 - val_loss: 0.1125
Epoch 11/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0210 - val_loss: 0.113

242/242 [==============================] - 1s 3ms/step - loss: 0.0241 - val_loss: 0.1108
Epoch 9/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0236 - val_loss: 0.1126
Epoch 10/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0231 - val_loss: 0.1114
Epoch 11/50
242/242 [==============================] - 1s 2ms/step - loss: 0.0231 - val_loss: 0.1135
Epoch 12/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0226 - val_loss: 0.1097
Epoch 13/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0222 - val_loss: 0.1062
Epoch 14/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0219 - val_loss: 0.1081
Epoch 15/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0219 - val_loss: 0.1069
Epoch 16/50
242/242 [==============================] - 1s 2ms/step - loss: 0.0218 - val_loss: 0.1077
Epoch 17/50
242/242 [==============================] - 1s 2ms/step - loss: 0.0214 - val_loss: 0.1101
Epo

242/242 [==============================] - 1s 3ms/step - loss: 0.0213 - val_loss: 0.1003
Epoch 11/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0211 - val_loss: 0.1021
Epoch 12/50
242/242 [==============================] - 1s 2ms/step - loss: 0.0207 - val_loss: 0.1015
Epoch 13/50
242/242 [==============================] - 1s 2ms/step - loss: 0.0205 - val_loss: 0.1038
Epoch 14/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0203 - val_loss: 0.0992
Epoch 15/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0200 - val_loss: 0.1018
Epoch 16/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0198 - val_loss: 0.1009
Epoch 17/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0194 - val_loss: 0.1003
Epoch 18/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0194 - val_loss: 0.0989
Epoch 19/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0192 - val_loss: 0.0989
Ep

242/242 [==============================] - 1s 3ms/step - loss: 0.0262 - val_loss: 0.1123
Epoch 5/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0247 - val_loss: 0.1104
Epoch 6/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0241 - val_loss: 0.1121
Epoch 7/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0230 - val_loss: 0.1055
Epoch 8/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0222 - val_loss: 0.1086
Epoch 9/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0216 - val_loss: 0.1049
Epoch 10/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0212 - val_loss: 0.1052
Epoch 11/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0207 - val_loss: 0.1021
Epoch 12/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0204 - val_loss: 0.1000
Epoch 13/50
242/242 [==============================] - 1s 3ms/step - loss: 0.0203 - val_loss: 0.1018
Epoch 1

In [43]:
model.build(input_shape=(x_train.shape[1],))
model.summary()

Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_165 (Dense)           (None, 31)                992       
                                                                 
 dense_153 (Dense)           (None, 10)                320       
                                                                 
Total params: 1,312
Trainable params: 1,312
Non-trainable params: 0
_________________________________________________________________


In [46]:
best_loss

0.08281312882900238

In [52]:
for k,v in test_model_architecture_losses.items():
        print(k,v)

50_ 0.1431264877319336
10_ 0.16071981191635132
20_ 0.11370912194252014
35_ 0.17247316241264343
10_10_ 0.1164359450340271
20_10_ 0.1020575612783432
35_10_ 0.10604863613843918
50_10_ 0.1025320440530777
10_20_ 0.09640335291624069
20_20_ 0.12313917279243469
35_20_ 0.1177125945687294
50_20_ 0.09784546494483948
10_35_ 0.09982040524482727
20_35_ 0.11052022874355316
35_35_ 0.10480693727731705
50_35_ 0.10040464252233505
10_50_ 0.10938628017902374
20_50_ 0.0980762466788292
35_50_ 0.11215102672576904
50_50_ 0.10253310948610306
10_10_10_ 0.08828307688236237
20_10_10_ 0.11525263637304306
35_10_10_ 0.1047535091638565
50_10_10_ 0.10384004563093185
10_20_10_ 0.10881024599075317
20_20_10_ 0.095396488904953
35_20_10_ 0.1020490825176239
50_20_10_ 0.09817603975534439
10_35_10_ 0.0941237285733223
20_35_10_ 0.10400105267763138
35_35_10_ 0.1054578348994255
50_35_10_ 0.09550423920154572
10_50_10_ 0.09918875992298126
20_50_10_ 0.09983114898204803
35_50_10_ 0.0919569581747055
50_50_10_ 0.1001627966761589
10_10_